notebook used to test the training of MCD models

In [2]:
import datetime
import os
import time

import uncertainty_wizard as uwiz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from config import Config
from self_driving_car_batch_generator import Generator
from utils import get_driving_styles
from utils import INPUT_SHAPE
from utils_models import *
from self_driving_car_train import *

np.random.seed(0)

In [3]:
cfg = Config()
cfg.from_pyfile("config_my.py")

True

In [5]:
cfg.DROPOUT_RATE

0.0

In [ ]:
cfg.DROPOUT_RATE = 0.05

In [ ]:
def build_model(model_name, rate, use_dropout=False):
    """
    Retrieve the DAVE-2 NVIDIA model
    """
    model = None
    if "dave2" in model_name:
        model = create_dave2_model(rate,use_dropout)
    else:
        print("Incorrect model name provided")
        exit()

    assert model is not None
    model.summary()

    return model

In [ ]:
def create_dave2_model(drop_rate, use_dropout=False):
    """
    Modified NVIDIA model w/ Dropout layers
    """
    if use_dropout:
        rate = drop_rate
        inputs = keras.Input(shape=INPUT_SHAPE)
        lambda_layer = keras.layers.Lambda(lambda x: x / 127.5 - 1.0, name="lambda_layer")(inputs)
        x = keras.layers.Conv2D(24, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=l2(1.0e-6))(
            lambda_layer)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Conv2D(36, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Conv2D(48, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Dense(50, activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        x = keras.layers.Dense(10, activation='relu', kernel_regularizer=l2(1.0e-6))(x)
        x = keras.layers.Dropout(rate)(x, training=True)
        outputs = keras.layers.Dense(1)(x)
        model = keras.Model(inputs=inputs, outputs=outputs)
    else:
        """
        original NVIDIA model w/out Dropout layers
        """
        model = Sequential()
        model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=INPUT_SHAPE))
        model.add(Conv2D(24, (5, 5), activation='elu', strides=(2, 2)))
        model.add(Conv2D(36, (5, 5), activation='elu', strides=(2, 2)))
        model.add(Conv2D(48, (5, 5), activation='elu', strides=(2, 2)))
        model.add(Conv2D(64, (3, 3), activation='elu'))
        model.add(Conv2D(64, (3, 3), activation='elu'))
        model.add(Dropout(rate=0.05))
        model.add(Flatten())
        model.add(Dense(100, activation='elu'))
        model.add(Dense(50, activation='elu'))
        model.add(Dense(10, activation='elu'))
        model.add(Dense(1))

    return model


In [ ]:
x_train, x_test, y_train, y_test = load_data(cfg)

In [ ]:
for i in range(5,100,5):
    r = i/100
    rs= (str)(r)
    model = build_model(cfg.SDC_MODEL_NAME, r ,cfg.USE_PREDICTIVE_UNCERTAINTY)
    train_model(model, cfg, x_train, x_test, y_train, y_test)
    name = os.path.join(cfg.SDC_MODELS_DIR,
                            cfg.TRACK + '-' + rs +'-MCD.h5')
    model.save(name)